In [78]:
import re
import pandas as pd
import nltk

from nltk.tree import Tree
# This uses corenlp server! Will need to alter code if using JAR files directly
# https://stanfordnlp.github.io/CoreNLP/corenlp-server.html
from nltk.parse.corenlp import CoreNLPParser
from nltk.tag.stanford import CoreNLPTagger, CoreNLPPOSTagger, CoreNLPNERTagger

import numpy as np
from nltk.corpus import wordnet as wn
from nltk.metrics import edit_distance as ed
from enchant import Dict as dictionary

essay_key = pd.read_csv('D:/UIC/Spring 2018/NLP/Project/essay_split.csv', sep=',')
essay_key.head()

,filename,grade,word_len,grader
0,990384.txt,high,568,Aldo
1,395987.txt,high,508,John
2,1949465.txt,high,458,Aldo
3,38209.txt,high,456,John
4,1834502.txt,high,454,Aldo


In [79]:
essays = []
for filename in essay_key['filename']:
    with open('D:/UIC/Spring 2018/NLP/Project/essays_dataset/essays/'+filename, 'r') as f:
        essays.append(f.read().strip())
        
essay_key['essay'] = essays
essay_key.head()

,filename,grade,word_len,grader,essay
0,990384.txt,high,568,Aldo,I dont think that in twenty years ther will be...
1,395987.txt,high,508,John,I disagree with the statement. Lets guess what...
2,1949465.txt,high,458,Aldo,I agree that twenty from now there will be few...
3,38209.txt,high,456,John,It is generally believed that young people enj...
4,1834502.txt,high,454,Aldo,"There is a famous saying which reads: ""Give a ..."


In [80]:
matches = wn.synsets('Dog')
len(matches)

8

In [81]:
def pos_tags(tagger, sentences):
    """Tags sentences with POS tags. Returns a list of (word, tag) tuples

    tagger (CoreNLPTagger): a tagger to tag sentences
    RETURNS (list): list of (word, tag) tuples
    """
    default_properties = {'annotators': 'tokenize,ssplit,pos'}
    tagged_data = tagger.api_call(sentences, properties=default_properties)
    
    tags = list()
    for sent in tagged_data['sentences']:
        #modified to return the start index and end index of the word
        tags.append([(token['word'], token['pos'], token['characterOffsetBegin'], token['characterOffsetEnd']) for token in sent['tokens']])
    return tags

In [82]:
tagger = CoreNLPPOSTagger(url='http://localhost:9000')

In [83]:
result = pos_tags(tagger, essay_key.loc[0,'essay'])
result

[[('I', 'PRP', 0, 1),
  ('dont', 'VBP', 2, 6),
  ('think', 'VB', 7, 12),
  ('that', 'IN', 13, 17),
  ('in', 'IN', 18, 20),
  ('twenty', 'CD', 21, 27),
  ('years', 'NNS', 28, 33),
  ('ther', 'RB', 34, 38),
  ('will', 'MD', 39, 43),
  ('be', 'VB', 44, 46),
  ('fewer', 'JJR', 47, 52),
  ('cars', 'NNS', 53, 57),
  ('than', 'IN', 58, 62),
  ('now', 'RB', 63, 66),
  ('because', 'IN', 67, 74),
  ('sales', 'NNS', 75, 80),
  ('of', 'IN', 81, 83),
  ('cars', 'NNS', 84, 88),
  ('are', 'VBP', 89, 92),
  ('increasing', 'VBG', 93, 103),
  ('day', 'NN', 104, 107),
  ('by', 'IN', 108, 110),
  ('day', 'NN', 111, 114),
  ('.', '.', 114, 115)],
 [('More', 'JJR', 116, 120),
  ('people', 'NNS', 121, 127),
  ('are', 'VBP', 128, 131),
  ('using', 'VBG', 132, 137),
  ('cars', 'NNS', 138, 142),
  ('as', 'IN', 143, 145),
  ('it', 'PRP', 146, 148),
  ('is', 'VBZ', 149, 151),
  ('easy', 'JJ', 152, 156),
  ('to', 'TO', 157, 159),
  ('own', 'VB', 160, 163),
  ('a', 'DT', 164, 165),
  ('car', 'NN', 166, 169),
  ('to

In [84]:
#https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return ''

In [85]:
for sent in result:
    for pos_t in sent:
        matches = wn.synsets(pos_t[0])
        
        print(pos_t[0], pos_t[1], get_wordnet_pos(pos_t[1]),len(matches))

I PRP  4
dont VBP v 0
think VB v 14
that IN  0
in IN  7
twenty CD  3
years NNS n 7
ther RB r 0
will MD  6
be VB v 14
fewer JJR a 2
cars NNS n 5
than IN  0
now RB r 8
because IN  0
sales NNS n 6
of IN  0
cars NNS n 5
are VBP v 14
increasing VBG v 4
day NN n 10
by IN  2
day NN n 10
. .  0
More JJR a 5
people NNS n 6
are VBP v 14
using VBG v 7
cars NNS n 5
as IN  10
it PRP  1
is VBZ v 13
easy JJ a 15
to TO  0
own VB v 2
a DT  7
car NN n 5
today NN n 4
and CC  0
many JJ a 1
low JJ a 16
cost NN n 5
cars NNS n 5
are VBP v 14
coming VBG v 26
into IN  0
the DT  0
depending VBG v 2
on IN  5
demand NN n 11
of IN  0
people NNS n 6
ans NNS n 2
usage NN n 3
. .  0
Twenty CD  3
years NNS n 7
from IN  0
now RB r 8
there EX  4
will MD  6
be VB v 14
more RBR r 5
and CC  0
more RBR r 5
people NNS n 6
coming VBG v 26
to TO  0
cities NNS n 3
and CC  0
towns NNS n 4
in IN  7
search NN n 9
of IN  0
jobs NNS n 17
and CC  0
other JJ a 4
requirements NNS n 3
and CC  0
population NN n 5
is VBZ v 13
increasing V

In [86]:
wn.synsets('that')

[]

In [87]:
chkr = SpellChecker("en_US", essay_key.loc[0, 'essay'])
chkr

In [105]:
def check_spelling(dictionary, sentences):
    """Check the spelling of tagged words on an essay
    and return the list of misspelling words with their tags 
    and indexes of begining and end of those words"""
    wrong_words = []
    for sentence in sentences:
        for w_tuple in sentence:
            word = w_tuple[0]
            if(word !=',' and word != "'s"):
                if dicc.check(word) is False:
                    tag = w_tuple[1]
                    b_pos = w_tuple[2]
                    e_pos = w_tuple[3]
                    wrong_words.append((word, tag, b_pos, e_pos))
    return wrong_words

In [92]:
dicc = dictionary("en_US")

In [93]:
result = check_spelling(dicc, result)

In [94]:
result

[('dont', 'VBP', 2, 6),
 ('ther', 'RB', 34, 38),
 ('becuase', 'NN', 533, 540),
 ('becuase', 'NN', 541, 548),
 ('donot', 'VBP', 567, 572),
 ('dont', 'VBP', 640, 644),
 ('dont', 'VBP', 795, 799),
 (',', ',', 1055, 1056),
 ('increains', 'NNS', 1139, 1148),
 ('increaing', 'VBG', 1450, 1459),
 (',', ',', 1541, 1542),
 ('devloped', 'VBN', 1568, 1576),
 ("'s", 'POS', 2056, 2058),
 ('eard', 'VB', 2069, 2073),
 ('mediun', 'NN', 2332, 2338),
 (',', ',', 2518, 2519),
 ('donot', 'JJ', 2639, 2644),
 ('disturbace', 'NN', 2659, 2669),
 ('dont', 'VBP', 2802, 2806),
 ('varities', 'NNS', 2916, 2924)]

In [95]:
tagged_sentences = pos_tags(tagger, essay_key.loc[0,'essay'])
result = check_spelling(dicc, tagged_sentences)
result

[('dont', 'VBP', 2, 6),
 ('ther', 'RB', 34, 38),
 ('becuase', 'NN', 533, 540),
 ('becuase', 'NN', 541, 548),
 ('donot', 'VBP', 567, 572),
 ('dont', 'VBP', 640, 644),
 ('dont', 'VBP', 795, 799),
 (',', ',', 1055, 1056),
 ('increains', 'NNS', 1139, 1148),
 ('increaing', 'VBG', 1450, 1459),
 (',', ',', 1541, 1542),
 ('devloped', 'VBN', 1568, 1576),
 ("'s", 'POS', 2056, 2058),
 ('eard', 'VB', 2069, 2073),
 ('mediun', 'NN', 2332, 2338),
 (',', ',', 2518, 2519),
 ('donot', 'JJ', 2639, 2644),
 ('disturbace', 'NN', 2659, 2669),
 ('dont', 'VBP', 2802, 2806),
 ('varities', 'NNS', 2916, 2924)]

In [106]:
tagger = CoreNLPPOSTagger(url='http://localhost:9000')
dicc = dictionary("en_US")

misspelling_words = list()
misspelling_count = list()
for essay in essay_key['essay']:
    tagged_sentences = pos_tags(tagger, essay)
    checked_words = check_spelling(dicc, tagged_sentences)
    misspelling_words.append(checked_words)
    misspelling_count.append(len(checked_words))

essay_key['misspelling'] = misspelling_words
essay_key['misspelling_count'] = misspelling_count

essay_key
    

,filename,grade,word_len,grader,essay,misspelling,misspelling_count
0,990384.txt,high,568,Aldo,I dont think that in twenty years ther will be...,"[(dont, VBP, 2, 6), (ther, RB, 34, 38), (becua...",16
1,395987.txt,high,508,John,I disagree with the statement. Lets guess what...,"[(everthing, NN, 644, 653), (aeroplane, NN, 11...",6
2,1949465.txt,high,458,Aldo,I agree that twenty from now there will be few...,"[(non-renewable, JJ, 406, 419), (littel, NN, 4...",11
3,38209.txt,high,456,John,It is generally believed that young people enj...,"[(n't, RB, 533, 536), (``, ``, 759, 760), ('',...",13
4,1834502.txt,high,454,Aldo,"There is a famous saying which reads: ""Give a ...","[(:, :, 36, 37), (``, ``, 38, 39), ('', '', 16...",9
5,1079196.txt,high,430,John,"Personally, I agree with the statement saying ...","[(\, VBP, 406, 407), (advertisemetns, NNS, 538...",7
6,1985615.txt,high,426,Aldo,I think understanding ideas and concepts is mo...,"[(n't, RB, 974, 977), (n't, RB, 1260, 1263), (...",11
7,711394.txt,high,424,John,The use of cars has increased rapidly over the...,"[(dirve, VBP, 502, 507), (Allthough, NNP, 1140...",4
8,1728162.txt,high,419,Aldo,Swamped with commercials and advertisments in ...,"[(advertisments, NNS, 29, 42), (advertisment, ...",21
9,186616.txt,high,403,John,"Since the ancient times, man has investigated ...","[(privilage, NN, 352, 361), (belive, VBP, 1052...",3


In [104]:
essay_key[['grade','essay','misspelling','misspelling_count']].to_csv('D:/UIC/Spring 2018/NLP/Project/spelling_count.csv', index=False)